In [141]:
pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [148]:
import simpy
import random
from scipy.stats import bernoulli

carril_pesado = 1
carril_liviano = 2
intervalo_t_llegada = 3
porcentaje_carro_pesado = 0.15
x_revision = 20
desviacion = 10


porcentaje_carro_sin_notificar = 0.60
tiempo_revision = 2 * 24 * 60

class Automovil:
    def __init__(self, tipo: int):
        self.tipo_automovil = tipo
        self.estado = None
        self.tiempo_arreglado = 0

    def __str__(self):
        return f'tipo: {self.tipo_automovil}, estado {self.estado}'

def generar_automovil():
   bernoulli.rvs(porcentaje_carro_pesado)

def generar_estacionamiento():
   random.randint(3, 5)

def generar_turno():
   random.randint(1, 3)

def generar_matricula():
   random.randint(5, 10)

def generar_paso_revision():
   random.normalvariate(mu = x_revision, sigma = desviacion)

def generar_paso_revision():
   bernoulli.rvs(porcentaje_carro_sin_notificar)

def generar_sello():
   random.randint(1, 3)

def generar_salida():
   random.randint(5, 10)


generar_automovil = lambda : bernoulli.rvs(porcentaje_carro_pesado)

generar_estacionamiento = lambda : random.randint(3, 5)
generar_turno = lambda  : random.randint(1, 3)
generar_matricula = lambda  : random.randint(5, 10)

generar_paso_revision = lambda : random.normalvariate(mu = x_revision, sigma = desviacion)

generar_notificacion = lambda : bernoulli.rvs(porcentaje_carro_sin_notificar)

generar_sello = lambda  : random.randint(1, 3)
generar_salida = lambda  : random.randint(5, 10)

class Emov:
    def __init__(self, environment: simpy.Environment):
        self.environment: simpy.Environment = environment
        self.carril_pesado: simpy.Resource = simpy.Resource(environment, carril_pesado)
        self.carril_normal: simpy.Resource = simpy.Resource(environment, carril_liviano)
        self.vehiculo: Automovil = None
        self.vehiculos: [dict[int: Automovil]] = []

    def matricular_automovil(self, auto: Automovil = None):
        self.vehiculo = auto

        yield self.environment.process(self.estacionar())
        yield self.environment.process(self.obtener_turno())
        yield self.environment.process(self.pagar_matriculas())
        yield self.environment.process(self.pasar_a_revision())


    def arrivar(self, tipo):

        self.vehiculo = Automovil(tipo=tipo)

        if self.vehiculo.tipo_automovil == 0: # Auto normal...
           return self.carril_normal.request()
        else: # Auto gran tonelaje...
            return self.carril_pesado.request()

    def estacionar(self):
        yield self.environment.timeout(generar_estacionamiento())

    def obtener_turno(self):
        yield self.environment.timeout(generar_turno())

    def pagar_matriculas(self):
        yield self.environment.timeout(generar_matricula())

    def pasar_a_revision(self):
        if self.vehiculo is None:
            carril = self.arrivar(generar_automovil())
        else:
            carril = self.arrivar(self.vehiculo.tipo_automovil)
        if carril.triggered:
            tiempo_revision = generar_paso_revision()
            while tiempo_revision < 0: # Evita que se genere valores negativos para el timeout.
                tiempo_revision = generar_paso_revision()
            yield self.environment.timeout(tiempo_revision)
            carril.resource.release(carril)
            notificacion = generar_notificacion()
            if notificacion == 1: # Sin notificacion
                yield self.environment.process(self.poner_sello())
                yield self.environment.process(self.salir())
                self.vehiculo.estado = 'aprobado'
                self.vehiculos.append((self.vehiculo, self.environment.now))
                self.vehiculo = None
            else:
                self.vehiculo.estado = 'renegado'
                self.vehiculo.tiempo_arreglado = self.environment.now + tiempo_revision
                self.vehiculos.append((self.vehiculo, self.environment.now))

    def poner_sello(self):
        yield self.environment.timeout(generar_sello())

    def salir(self):
        yield self.environment.timeout(generar_salida())

datos_por_dia = {}

def matricula(global_environment: simpy.Environment):
    global emov
    emov = Emov(environment=global_environment)
    while True:
        if emov.vehiculo is not None and emov.vehiculo.tiempo_arreglado > global_environment.now:
            yield environment.process(emov.matricular_automovil(emov.vehiculo))
        else:
            auto = Automovil(tipo=generar_automovil())
            yield environment.process(emov.matricular_automovil(auto))

random.seed(42)
emov: Emov = None
environment = simpy.Environment()
environment.process(matricula(environment))
environment.run(until=30*24*60)

dias_en_minutos = 24*60
segundos = list([x*24*60 for x in range(31)])
segundos.pop(0)

dia_actual = segundos[0]
resultados = dict()
lista = []

for i in emov.vehiculos:
    if i[1] <= dia_actual:
        lista.append(i[0].tipo_automovil)
    else:
        resultados[dia_actual] = lista
        dia_actual = segundos[segundos.index(dia_actual)+1]
        lista = list()

from collections import Counter

pasar_a_dias = lambda x : x/(24*60)
rst = list(map(pasar_a_dias, list(resultados.keys())))

new_resultados = dict(zip(rst, list(resultados.values())))

livianos_por_dia = {f'{int(key)}': Counter(values)[0] for key, values in new_resultados.items()}
pesados_por_dia  = {key: Counter(values)[1] for key, values in new_resultados.items()}


fig.update_layout(
    xaxis_title='Dias',
    yaxis_title='x')

dias_en_minutos = 24*60
segundos = list([x*24*60 for x in range(31)])
segundos.pop(0)

dia_actual = segundos[0]
resultados = dict()
lista = []

for i in emov.vehiculos:
    if i[1] <= dia_actual:
        lista.append(i[0].estado)
    else:
        resultados[dia_actual] = lista
        dia_actual = segundos[segundos.index(dia_actual)+1]
        lista = list()

from collections import Counter

pasar_a_dias = lambda x : x/(24*60)
rst = list(map(pasar_a_dias, list(resultados.keys())))

new_resultados = dict(zip(rst, list(resultados.values())))

aprobados_por_dia = {f'{int(key)}': Counter(values).get('aprobado') for key, values in new_resultados.items()}
rechazados_por_dia  = {key: Counter(values).get('renegado') for key, values in new_resultados.items()}



In [149]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

x_values=list(livianos_por_dia.keys())


fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_bar(name='Liviano',x=x_values, y=list(livianos_por_dia.values()), secondary_y=False)
fig.add_bar(name='Pesado', x=x_values, y=list(pesados_por_dia.values()), secondary_y=True)

fig.update_layout(
    xaxis_title='dia',
    yaxis_title='tipo')

In [147]:
x_values=list(livianos_por_dia.keys())

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_bar(name='Aprovados',x=x_values, y=list(aprobados_por_dia.values()), secondary_y=False)

fig.add_bar(name='Renegados', x=x_values, y=list(rechazados_por_dia.values()), secondary_y=True)

fig.update_layout(
    xaxis_title='dia',
    yaxis_title='salida')